In [3]:
from pyspark.sql.functions import col,explode
from zoneinfo import ZoneInfo
from datetime import datetime,timedelta

StatementMeta(, a1c02116-6e53-401a-868c-ee8190f09cc9, 5, Finished, Available, Finished)

In [4]:
egypt_tz = ZoneInfo('Africa/Cairo')
yesterday = datetime.now(egypt_tz) - timedelta(days=1)
file_path = f"Files/youtube_trending_{yesterday.strftime('%Y-%m-%d')}.json"
df = spark.read.option("multiline", "true").json(file_path)
file_path_id=f"Files/youtube_trending_{yesterday.strftime('%Y-%m-%d')}_ID.json"
df_ID = spark.read.option("multiline", "true").json(file_path_id)


StatementMeta(, a1c02116-6e53-401a-868c-ee8190f09cc9, 6, Finished, Available, Finished)

In [5]:
df_exploded = df.select(explode("items").alias("item"))
df_exploded_id=df_ID.select(explode("items").alias("item"))


StatementMeta(, a1c02116-6e53-401a-868c-ee8190f09cc9, 7, Finished, Available, Finished)

In [6]:
df_clean = df_exploded.select(
    "item.id",  # video ID
    "item.snippet.channelId",
    "item.snippet.publishedAt",
    "item.snippet.channelTitle",
    "item.snippet.title",
    "item.statistics.viewCount",
    "item.statistics.likeCount",
    "item.statistics.commentCount"
)
display(df_clean)

StatementMeta(, a1c02116-6e53-401a-868c-ee8190f09cc9, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ad695369-c37e-4ad4-bff3-cdf0d577b448)

In [7]:
df_clean_id=df_exploded_id.select(
    col("item.id").alias("Channel_id"),
    "item.statistics.subscriberCount",
    "item.snippet.thumbnails.default.url",
)


StatementMeta(, a1c02116-6e53-401a-868c-ee8190f09cc9, 9, Finished, Available, Finished)

In [8]:
df_joined = df_clean.alias("vid").join(
    df_clean_id.alias("cid"),
    col("cid.channel_id") == col("vid.channelId"),
    how="left"
)
df_joined=df_joined.drop("channel_Id")


StatementMeta(, a1c02116-6e53-401a-868c-ee8190f09cc9, 10, Finished, Available, Finished)

In [9]:
# # Save to Lakehouse as a table
df_joined.write.mode("overwrite").saveAsTable(f"youtube_trending_{yesterday.strftime('%Y_%m_%d')}")


StatementMeta(, a1c02116-6e53-401a-868c-ee8190f09cc9, 11, Finished, Available, Finished)

AnalysisException: [DELTA_FAILED_TO_MERGE_FIELDS] Failed to merge fields 'viewCount' and 'viewCount'